# Lib

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob
import joblib

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

29/11/2022  02:35    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
29/11/2022  02:35         1.008.994 1.EA.ipynb
25/11/2022  01:27            74.828 2.Feature_engineering.ipynb
29/11/2022  03:09            28.642 3.Stability_var.ipynb
04/12/2022  14:59           170.765 4.Modeling.ipynb
29/11/2022  02:48            10.342 5.Modelo_analisis_negocio.ipynb
04/12/2022  17:40           167.928 6.Pipeline.ipynb
25/11/2022  01:32           156.224 7.Pipeline_SPARK.ipynb
24/11/2022  23:44    <DIR>          catboost_info
               7 arquivo(s)      1.617.723 bytes
               4 pasta(s)   371.847.122.944 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [4]:
df = pd.read_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet")

In [5]:
df.shape[0]

129413

In [6]:
seman = df.groupby("data").agg( 

    set_data = ("data",lambda x: list(np.unique(x)) ),
     total_dias = ("data",lambda x: len(list(np.unique(x))) ),
     total_um = ("fraude",lambda x: sum(x) ),
    total_zero = ("fraude",lambda x: sum( (x+1)%2 ) ),
    total = ("fraude","count" )
    
     )

seman['%_cusum'] = (seman['total']/seman['total'].sum()).cumsum()

seman.reset_index()

,data,set_data,total_dias,total_um,total_zero,total,%_cusum
0,2020-03-08,[2020-03-08T00:00:00.000000000],1,124,2714,2838,0.021930
1,2020-03-09,[2020-03-09T00:00:00.000000000],1,197,3976,4173,0.054175
2,2020-03-10,[2020-03-10T00:00:00.000000000],1,187,3918,4105,0.085896
3,2020-03-11,[2020-03-11T00:00:00.000000000],1,203,3696,3899,0.116024
4,2020-03-12,[2020-03-12T00:00:00.000000000],1,181,3606,3787,0.145287
5,2020-03-13,[2020-03-13T00:00:00.000000000],1,162,3067,3229,0.170238
6,2020-03-14,[2020-03-14T00:00:00.000000000],1,135,2425,2560,0.190020
7,2020-03-15,[2020-03-15T00:00:00.000000000],1,119,2651,2770,0.211424
8,2020-03-16,[2020-03-16T00:00:00.000000000],1,180,4022,4202,0.243894
9,2020-03-17,[2020-03-17T00:00:00.000000000],1,201,4079,4280,0.276966


In [7]:
colunas_orig_and_week = utils_ml.exclude_words( df.columns.tolist(), ["cat","bin","WoE","produto","fecha","data","hora","week","fraude"] ) + ['categoria_produto']

len(colunas_orig_and_week), len(df)

(17, 129413)

In [15]:
colunas_orig_and_week

['a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'pais',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'n_boolean',
 'o_obj',
 'p_boolean',
 'monto',
 'score',
 'categoria_produto']

## groups

In [9]:
## Ordenando as datas disponíveis no dataset
datas = np.sort( df['data'].unique() )
## total de dias para cada grupo
days = 10
## Total de grupos
total_groups = int(len(datas)/days) if int(len(datas)/days) == len(datas)/days else int(len(datas)/days+1)
## Lista de arrays contendo os dias para cada grupo
ls_group = []

i = 0
for x in range(1,total_groups+1):

    # print(x)
    group = datas[i:i+days]
    ls_group.append(group)

    i = i+days

print(f"Total de grupos:", total_groups)

Total de grupos: 4


In [10]:
ls_group

[array(['2020-03-08T00:00:00.000000000', '2020-03-09T00:00:00.000000000',
        '2020-03-10T00:00:00.000000000', '2020-03-11T00:00:00.000000000',
        '2020-03-12T00:00:00.000000000', '2020-03-13T00:00:00.000000000',
        '2020-03-14T00:00:00.000000000', '2020-03-15T00:00:00.000000000',
        '2020-03-16T00:00:00.000000000', '2020-03-17T00:00:00.000000000'],
       dtype='datetime64[ns]'),
 array(['2020-03-18T00:00:00.000000000', '2020-03-19T00:00:00.000000000',
        '2020-03-20T00:00:00.000000000', '2020-03-21T00:00:00.000000000',
        '2020-03-22T00:00:00.000000000', '2020-03-23T00:00:00.000000000',
        '2020-03-24T00:00:00.000000000', '2020-03-25T00:00:00.000000000',
        '2020-03-26T00:00:00.000000000', '2020-03-27T00:00:00.000000000'],
       dtype='datetime64[ns]'),
 array(['2020-03-28T00:00:00.000000000', '2020-03-29T00:00:00.000000000',
        '2020-03-30T00:00:00.000000000', '2020-03-31T00:00:00.000000000',
        '2020-04-01T00:00:00.000000000', '2020

# WoE by groups

In [81]:
label = "fraude"
dtype_ = 'categorical'
df_woe_ivs_ls = []

for n,variable in enumerate(colunas_orig_and_week, start=1):

    print(f"{n}/{len(colunas_orig_and_week)}",variable)

    ## carregando o modelo pretreinado de
    optb = joblib.load(f".\..\src\\features\\pipeline_binning__{variable}__.pkl")

    for n, group in enumerate(ls_group, start=1):

        df_filtred = df[df['data'].isin(group)][[variable,label]]
        df_filtred = utils_ml.encode_woe_var_transform(df_filtred, variable, optb )

        ## computting the WOE and IV to variable transformed with _cat in the end.
        df_iv = utils_ml.WOE_IV(df = df_filtred  , variable = f"{variable}_bins", dtype_ = dtype_ , label = label, monotonic= True, optimal_binning = False)
        df_iv['group'] = n


        df_woe_ivs_ls.append(df_iv)

df_woe_ivs = pd.concat(df_woe_ivs_ls).sort_values("IV_var", ascending=False)

1/17 a_int
2/17 b_float
3/17 c_float
4/17 d_float
5/17 e_float
6/17 f_float
7/17 pais
8/17 h_float
9/17 k_float
10/17 l_float
11/17 m_float
12/17 n_boolean
13/17 o_obj
14/17 p_boolean
15/17 monto
16/17 score
17/17 categoria_produto


In [82]:
df_woe_ivs['Bin'] = df_woe_ivs['Bin'].astype("string")
df_woe_ivs.query("Bin!='Missing' and Bin!='Special'").to_parquet(".\..\data\\iv_woe_by_week.parquet")

## Data to explore in powerBI

# 3. Desempenho


- Como você pode garantir que o desempenho do modelo no laboratório vai
ser um proxy para o desempenho do modelo em produção?



# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
